### ความคืบหน้าของโปรเจค

- ocr โดยใช้โมเดลที่เทรนมา

In [7]:
import pytesseract
from PIL import Image
from datetime import datetime
from difflib import SequenceMatcher

# กำหนดตำแหน่ง tesseract ในระบบ
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# ฟังก์ชันสำหรับการจับคู่ข้อความที่ดีที่สุด
def get_best_alignment(ref, ocr):
    matcher = SequenceMatcher(None, ref, ocr)
    matches = matcher.get_opcodes()
    return matches

# ฟังก์ชันสำหรับการประเมินการจับคู่ข้อความ
def evaluate_alignment(matches, ref, ocr):
    correct_chars = 0
    incorrect_chars = 0
    errors = []
    correct_details = []

    ref_chars = list(ref)
    ocr_chars = list(ocr)

    for tag, i1, i2, j1, j2 in matches:
        if tag == 'equal':
            correct_chars += (i2 - i1)
            for i in range(i1, i2):
                correct_details.append((i, ref_chars[i]))
        else:
            incorrect_chars += max(i2 - i1, j2 - j1)
            for i in range(i1, i2):
                expected_char = ref_chars[i]
                got_char = ocr_chars[j1 + (i - i1)] if j1 + (i - i1) < len(ocr_chars) else 'None'
                errors.append((i, expected_char, got_char))

    return correct_chars, incorrect_chars, errors, correct_details

# กำหนดเส้นทางของไฟล์ภาพและไฟล์อ้างอิง
# image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/easy2jpg.jpg'
# reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/testfile/easyreal1.txt'

# image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/med1jpg.jpg'
# reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/testfile/mediumreal1.txt'

image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/file1.jpg'
reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/results/FORTEST.txt'

# เปิดภาพและทำการแปลงเป็นโทนสีเทา
image = Image.open(image_path)
gray_image = image.convert('L')

# กำหนดการตั้งค่า Tesseract OCR
custom_config = r'--tessdata-dir "/Users/gunnviryasiri/pythainlp-data/tessdata" --oem 1 --psm 3'

# ดึงข้อความจากภาพด้วย Tesseract
ocr_text = pytesseract.image_to_string(gray_image, config=custom_config, lang='tha+eng')

# ทำการลบช่องว่างและบรรทัดว่างออกจากข้อความ OCR
processed_ocr_text = "\n".join([line for line in ocr_text.splitlines() if line.strip()])

# สร้างไฟล์ผลลัพธ์พร้อมชื่อไฟล์ที่เป็นวันและเวลา
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_text_file = f'/Users/gunnviryasiri/Desktop/ocr/results/ocr_output_{current_time}.txt'

# บันทึกข้อความที่ได้จาก OCR ลงในไฟล์
with open(output_text_file, 'w', encoding='utf-8') as f:
    f.write(processed_ocr_text)

# เปิดไฟล์อ้างอิงและลบช่องว่างพิเศษออก
with open(reference_text_file, 'r', encoding='utf-8') as f:
    reference_text = f.read().replace("\n", "")

# ลบช่องว่างระหว่างตัวอักษรเพื่อจัดการปัญหาช่องว่าง
processed_ocr_text = processed_ocr_text.replace(" ", "").replace("\n", "")
reference_text = reference_text.replace(" ", "").replace("\n", "")

# ทำการจับคู่ข้อความ OCR กับไฟล์อ้างอิง
matches = get_best_alignment(reference_text, processed_ocr_text)

# ประเมินการจับคู่
correct_chars, incorrect_chars, errors, correct_details = evaluate_alignment(matches, reference_text, processed_ocr_text)

# คำนวณค่า Precision, Recall และ Accuracy
total_chars = len(reference_text)
precision = (correct_chars / (correct_chars + incorrect_chars)) * 100 if (correct_chars + incorrect_chars) > 0 else 0
recall = (correct_chars / total_chars) * 100 if total_chars > 0 else 0
accuracy = correct_chars / total_chars if total_chars > 0 else 0

# คำนวณ Edit Distance
edit_distance = sum([max(i2 - i1, j2 - j1) for tag, i1, i2, j1, j2 in matches if tag != 'equal'])

# แสดงผลลัพธ์ในรูปแบบ Confusion Matrix และข้อมูลอื่น ๆ
print("=== Confusion Matrix ===")
print(f"Total Characters in Reference Text: {total_chars}")
print(f"Correct Characters: {correct_chars}")
print(f"Incorrect Characters: {incorrect_chars}\n")

print("=== Metrics ===")
print(f"Final Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"Edit Distance: {edit_distance}\n")

print("=== Character Error Details ===")
for i, expected, got in errors:
    print(f"Position {i}: Expected '{expected}', Got '{got}'")

print("=== True Character Details ===")
for i, correct_char in correct_details:
    print(f"Position {i}: Correct '{correct_char}'")

print(f"\nText extracted, cleaned, and saved to {output_text_file}")


=== Confusion Matrix ===
Total Characters in Reference Text: 1981
Correct Characters: 1861
Incorrect Characters: 133

=== Metrics ===
Final Accuracy: 93.94%
Precision: 93.33%
Recall: 93.94%
Edit Distance: 133

=== Character Error Details ===
Position 0: Expected '!', Got 'ท'
Position 8: Expected '๗', Got '๕'
Position 12: Expected '๑', Got '๕'
Position 13: Expected '๘', Got 'ต'
Position 14: Expected '๘', Got 'ส'
Position 70: Expected '๘', Got '๕'
Position 123: Expected '๙', Got '๕'
Position 126: Expected '๖', Got '9'
Position 139: Expected '๖', Got '๐'
Position 140: Expected '๗', Got '๕'
Position 422: Expected '๒', Got '๐'
Position 423: Expected '๘', Got '๒'
Position 424: Expected '๖', Got '๕'
Position 428: Expected '๖', Got '๒'
Position 429: Expected '๖', Got '๒'
Position 437: Expected 'i', Got 'R'
Position 441: Expected 'e', Got 'o'
Position 451: Expected 'I', Got 'f'
Position 452: Expected 'ร', Got 'u'
Position 453: Expected 'ุ', Got 'M'
Position 454: Expected '่', Got 'i'
Position 4

- ocr โดยใช้ library ของมันเอง

#### ปัจจุบันแม่นกว่า

In [6]:
import pytesseract
from PIL import Image
from datetime import datetime
from difflib import SequenceMatcher

# กำหนดตำแหน่ง tesseract ในระบบ
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# ฟังก์ชันสำหรับการจับคู่ข้อความที่ดีที่สุด
def get_best_alignment(ref, ocr):
    matcher = SequenceMatcher(None, ref, ocr)
    matches = matcher.get_opcodes()
    return matches

# ฟังก์ชันสำหรับการประเมินการจับคู่ข้อความ
def evaluate_alignment(matches, ref, ocr):
    correct_chars = 0
    incorrect_chars = 0
    errors = []
    correct_details = []

    ref_chars = list(ref)
    ocr_chars = list(ocr)

    for tag, i1, i2, j1, j2 in matches:
        if tag == 'equal':
            correct_chars += (i2 - i1)
            for i in range(i1, i2):
                correct_details.append((i, ref_chars[i]))
        else:
            incorrect_chars += max(i2 - i1, j2 - j1)
            for i in range(i1, i2):
                expected_char = ref_chars[i]
                got_char = ocr_chars[j1 + (i - i1)] if j1 + (i - i1) < len(ocr_chars) else 'None'
                errors.append((i, expected_char, got_char))

    return correct_chars, incorrect_chars, errors, correct_details

# กำหนดเส้นทางของไฟล์ภาพและไฟล์อ้างอิง
# image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/easy2jpg.jpg'
# reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/testfile/easyreal1.txt'

# image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/med1jpg.jpg'
# reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/testfile/mediumreal1.txt'

image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/file1.jpg'
reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/results/FORTEST.txt'

# เปิดภาพและทำการแปลงเป็นโทนสีเทา
image = Image.open(image_path)
gray_image = image.convert('L')

# # กำหนดการตั้งค่า Tesseract OCR
# custom_config = r'--tessdata-dir "/Users/gunnviryasiri/pythainlp-data/tessdata" --oem 1 --psm 3'

# ดึงข้อความจากภาพด้วย Tesseract
ocr_text = pytesseract.image_to_string(gray_image, lang='tha+eng')

# ทำการลบช่องว่างและบรรทัดว่างออกจากข้อความ OCR
processed_ocr_text = "\n".join([line for line in ocr_text.splitlines() if line.strip()])

# สร้างไฟล์ผลลัพธ์พร้อมชื่อไฟล์ที่เป็นวันและเวลา
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_text_file = f'/Users/gunnviryasiri/Desktop/ocr/results/ocr_output_{current_time}.txt'

# บันทึกข้อความที่ได้จาก OCR ลงในไฟล์
with open(output_text_file, 'w', encoding='utf-8') as f:
    f.write(processed_ocr_text)

# เปิดไฟล์อ้างอิงและลบช่องว่างพิเศษออก
with open(reference_text_file, 'r', encoding='utf-8') as f:
    reference_text = f.read().replace("\n", "")

# ลบช่องว่างระหว่างตัวอักษรเพื่อจัดการปัญหาช่องว่าง
processed_ocr_text = processed_ocr_text.replace(" ", "").replace("\n", "")
reference_text = reference_text.replace(" ", "").replace("\n", "")

# ทำการจับคู่ข้อความ OCR กับไฟล์อ้างอิง
matches = get_best_alignment(reference_text, processed_ocr_text)

# ประเมินการจับคู่
correct_chars, incorrect_chars, errors, correct_details = evaluate_alignment(matches, reference_text, processed_ocr_text)

# คำนวณค่า Precision, Recall และ Accuracy
total_chars = len(reference_text)
precision = (correct_chars / (correct_chars + incorrect_chars)) * 100 if (correct_chars + incorrect_chars) > 0 else 0
recall = (correct_chars / total_chars) * 100 if total_chars > 0 else 0
accuracy = correct_chars / total_chars if total_chars > 0 else 0

# คำนวณ Edit Distance
edit_distance = sum([max(i2 - i1, j2 - j1) for tag, i1, i2, j1, j2 in matches if tag != 'equal'])

# แสดงผลลัพธ์ในรูปแบบ Confusion Matrix และข้อมูลอื่น ๆ
print("=== Confusion Matrix ===")
print(f"Total Characters in Reference Text: {total_chars}")
print(f"Correct Characters: {correct_chars}")
print(f"Incorrect Characters: {incorrect_chars}\n")

print("=== Metrics ===")
print(f"Final Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"Edit Distance: {edit_distance}\n")

print("=== Character Error Details ===")
for i, expected, got in errors:
    print(f"Position {i}: Expected '{expected}', Got '{got}'")

print("=== True Character Details ===")
for i, correct_char in correct_details:
    print(f"Position {i}: Correct '{correct_char}'")

print(f"\nText extracted, cleaned, and saved to {output_text_file}")


=== Confusion Matrix ===
Total Characters in Reference Text: 1981
Correct Characters: 1868
Incorrect Characters: 130

=== Metrics ===
Final Accuracy: 94.30%
Precision: 93.49%
Recall: 94.30%
Edit Distance: 130

=== Character Error Details ===
Position 0: Expected '!', Got 'ท'
Position 8: Expected '๗', Got '๕'
Position 12: Expected '๑', Got '๕'
Position 13: Expected '๘', Got 'ต'
Position 14: Expected '๘', Got 'ส'
Position 70: Expected '๘', Got '๕'
Position 123: Expected '๙', Got '๕'
Position 126: Expected '๖', Got '9'
Position 139: Expected '๖', Got '๐'
Position 140: Expected '๗', Got '๕'
Position 422: Expected '๒', Got '๐'
Position 423: Expected '๘', Got '๒'
Position 424: Expected '๖', Got '๕'
Position 428: Expected '๖', Got '๒'
Position 429: Expected '๖', Got '๒'
Position 437: Expected 'i', Got 'R'
Position 441: Expected 'e', Got 'o'
Position 449: Expected 'D', Got 'O'
Position 450: Expected 'J', Got 'u'
Position 451: Expected 'I', Got 'ร'
Position 479: Expected 'F', Got '6'
Position 4